<h1>End fo Life OS and COM creation</h1>

In [ ]:
# Data Shaping
import pandas as pd
import datetime as dt

# File Handling and Export
import json

# Data Visualization
import seaborn as sns
import matplotlib.pyplot as plt


# Web Builder
from jinja2 import Environment, FileSystemLoader

from ts_api_connections import datto_rmm_api


from flask import Blueprint, request, render_template, redirect, make_response

import pdfkit



# add current timestamp to filename for reference
current_time = (dt.datetime.utcnow().strftime('%Y_%m_%d_%H%M%S'))


In [ ]:

datto = datto_rmm_api.DattoRMM()
df_devices = datto.create_devices_dataframe()

In [ ]:
sites = df_devices['siteName'].unique().tolist()
print(sites[0])

In [ ]:
# 2. Create a template Environment
env = Environment(loader=FileSystemLoader(r'D:\Git\example_dev_flask_app\projects\inventory_control\templates'))

In [ ]:
# 3. Load the template from the Environment
template = env.get_template('reports/end_of_life_report.html')


In [ ]:
ventura_mask = df_devices["siteName"] == 'Example CU'

end_of_life_mask = (df_devices['isEndOfSupport'] == 1) | (df_devices['isNearEndOfSupport'] == 1)

df_ventura = df_devices[ventura_mask & end_of_life_mask]

df_columns = ['hostname', 'intIpAddress', 'operatingSystem', 'osEndOfSupportDate', 'lastLoggedInUser']
df_columns_rename = {
    "hostname": "hostname",
    "intIpAddress": "ip address",
    "operatingSystem": "operating system",
    "osEndOfSupportDate": "last day of support",
    "lastLoggedInUser": "last logged in user"
}

df_to_html = df_ventura[df_columns].sort_values('operatingSystem').rename(df_columns_rename, axis=1).fillna(
    "Unknown").to_html(index=False,
                       justify='unset')

df_ventura[df_columns].to_html(open('d:/exports/my_file.html', 'w'), index=False)

site_name = df_ventura['siteName'].unique()[0]

category_counts = df_ventura.value_counts('category').to_dict()

wkhtml_path = pdfkit.configuration(
    wkhtmltopdf="C:/Program Files/wkhtmltopdf/bin/wkhtmltopdf.exe")  # by using configuration you can add path value.


In [ ]:
# 4. Render the template with variables
html = template.render(title='OS End of Life Report',
                           site_name=site_name,
                             current_time=current_time,
                           category_counts=category_counts,
                           df=df_to_html)

In [ ]:
html

In [ ]:
# 5. Write the template to an HTML file
with open('d:/exports/html_report_jinja.html', 'w') as f:
    f.write(html)

In [ ]:

current_time = (dt.datetime.utcnow().strftime('%Y/%m/%d %H:%M:%S'))

reports_blueprint = Blueprint('reports', __name__, template_folder='templates', static_folder='static')




# 3. Load the template from the Environment
template = env.get_template('report_template.html')

# 4. Render the template with variables
html = template.render(title=title)

# 5. Write the template to an HTML file
with open('html_report_jinja.html', 'w') as f:
    f.write(html)

In [ ]:
df_devices.to_csv('.csv',index=False)

In [ ]:
fig = plt.figure(figsize=(60,40))
palette = sns.color_palette("rainbow")
sns.countplot(data=df_devices[df_devices['isNearEndOfSupport'] == 1].sort_values('siteName'),x = 'siteName',hue='operatingSystem')
plt.savefig(".png")

In [ ]:
df_devices['isEndOfSupport'] == 1

In [ ]:
end_of_life_mask = (df_devices['isEndOfSupport'] == 1)  | (df_devices['isNearEndOfSupport'] == 1)

In [ ]:
fig = plt.figure(figsize=(60,40))
palette = sns.color_palette("rainbow")
sns.countplot(data=(df_devices[end_of_life_mask].sort_values('siteName')),x = 'siteName',hue='isEndOfSupport')
plt.savefig(".png")

In [ ]:
ventura_mask = df_devices["siteName"] == 'Example CU'

In [ ]:
end_of_life_mask = (df_devices['isEndOfSupport'] == 1)  | (df_devices['isNearEndOfSupport'] == 1)

In [ ]:
df_devices[ventura_mask & end_of_life_mask]

In [ ]:
fig = plt.figure(figsize=(60,40))
palette = sns.color_palette("rainbow")
sns.countplot(data=df_ventura[df_ventura['isEndOfSupport'] == 1].sort_values('hostname'),x = 'operatingSystem',hue='offline30Days')
# plt.savefig(".png")

In [ ]:
df_devices = pd.read_excel(r"D:\Git\example_dev_flask_app\projects\inventory_control\static\seeds\end_of_life_finished.xlsx")

ventura_mask = df_devices["siteName"] == 'Example CU'

end_of_life_mask = (df_devices['isEndOfSupport'] == 1) | (df_devices['isNearEndOfSupport'] == 1)

df_ventura = df_devices[ventura_mask & end_of_life_mask]

df_columns = ['hostname', 'intIpAddress', 'operatingSystem', 'lastLoggedInUser']
df_columns_rename = {
    "hostname": "hostname",
    "intIpAddress": "ip address",
    "operatingSystem": "operating system",
    "lastLoggedInUser": "last logged in user"
}

df_to_html = df_ventura[df_columns].rename(df_columns_rename, axis=1).fillna("Unknown").to_html(index=False, justify='unset')


In [ ]:
df_ventura.value_counts('category').to_dict()

In [ ]:
import pdfkit

In [ ]:
pdfkit.from_url('https://google.com', 'd:/exports/out.pdf')

In [ ]:
import pdfkit
# from pdfkit.api import configuration

wkhtml_path = pdfkit.configuration(wkhtmltopdf = "C:/Program Files/wkhtmltopdf/bin/wkhtmltopdf.exe")  #by using configuration you can add path value.

pdfkit.from_url('http://[REDACTED_IP]:5000/reports/end-of-life', 'd:/exports/eol3.pdf', configuration = wkhtml_path)